In [1]:
import sportsdataverse.nfl as nfl 
from bs4 import BeautifulSoup
#import urllib.request
import requests
import pandas as pd
import numpy as np
import scipy
import difflib
import json
#import matplotlib.pyplot as plt
import time
import scipy
import statistics as stats
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt





In [2]:
# --------------------------- load and prepare data -------------------- #
pbp = nfl.load_nfl_pbp(seasons=([2024]),return_as_pandas=True)
data=pbp.copy()

pbp = pbp[((pbp['pass'] == 1) | (pbp['rush'] == 1)) & ( pbp.season_type == "REG")]
mask = {'home':1,'away':0}
pbp.posteam_type = pbp.posteam_type.map(mask)


cols = pbp.select_dtypes(include = 'number').columns
df_off = pbp.groupby(['posteam','game_id'])[cols].agg(['min','max','sum','median','mean','size'])
df_def = pbp.groupby(['defteam','game_id'])[cols].agg(['min','max','sum','median','mean','size'])


df_off = pbp.groupby(['posteam'])[cols].agg(['min','max','sum','median','mean','size'])
df_def = pbp.groupby(['defteam'])[cols].agg(['min','max','sum','median','mean','size'])

df_off.index = np.array(df_off.index) + '_off'
df_def.index = np.array(df_def.index) + '_def'

df = pd.concat([df_off,df_def])
df[[('pass_ypa','value')]] =  df[[('passing_yards','sum')]].values/df[[('pass_attempt','sum')]].values
df[[('rush_ypa','value')]] =  df[[('rushing_yards','sum')]].values/df[[('rush_attempt','sum')]].values
df[[('success_rate','value')]] =  df[[('success','sum')]].values/df[[('success','size')]].values

df = df.sort_index()

df_off = df.loc[df.index.str.contains('_off')]
df_def = df.loc[df.index.str.contains('_def')]



100%|██████████| 1/1 [00:00<00:00,  1.92it/s]


2      -45.676103
3       42.334431
4       57.355690
5      -22.536427
6      -42.409936
          ...    
8184     1.047212
8185     1.064670
8186     1.063710
8187    48.090810
8188   -57.440335
Name: pass_oe, Length: 6059, dtype: double[pyarrow]

In [51]:
# --------------------------- offense rank quick view -------------------- #
df_off[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value'),('pass_oe','mean')]].rank(ascending=False).astype('int').style.background_gradient(cmap='Blues')


,epa,passing_yards,pass_ypa,rush_ypa,success_rate,pass_oe
,mean,sum,value,value,value,mean
ARI_off,8,18,12,6,12,18
ATL_off,20,20,11,15,13,26
BAL_off,5,11,6,1,6,28
BUF_off,1,16,1,20,1,17
CAR_off,29,28,25,24,31,20
CHI_off,28,19,30,31,29,10
CIN_off,4,9,18,13,3,2
CLE_off,32,29,32,18,32,3
DAL_off,21,1,22,29,28,12


In [46]:
# --------------------------- defense rank quick view -------------------- #
df_def[[('epa','mean'),('passing_yards','sum'),('pass_ypa','value'),('rush_ypa','value'),('success_rate','value')]].rank(ascending=True).astype('int').style.background_gradient(cmap='Blues')


,epa,passing_yards,pass_ypa,rush_ypa,success_rate
,mean,sum,value,value,value
ARI_def,18,18,31,6,32
ATL_def,20,8,8,10,24
BAL_def,25,32,26,1,15
BUF_def,7,11,2,17,9
CAR_def,27,14,23,25,26
CHI_def,4,7,6,22,3
CIN_def,30,4,19,12,27
CLE_def,14,16,12,16,12
DAL_def,28,13,25,32,14


In [31]:
df_off.columns

MultiIndex([( 'play_id',    'min'),
            ( 'play_id',    'max'),
            ( 'play_id',    'sum'),
            ( 'play_id', 'median'),
            ( 'play_id',   'mean'),
            ( 'play_id',   'size'),
            (    'week',    'min'),
            (    'week',    'max'),
            (    'week',    'sum'),
            (    'week', 'median'),
            ...
            (   'xpass', 'median'),
            (   'xpass',   'mean'),
            (   'xpass',   'size'),
            ( 'pass_oe',    'min'),
            ( 'pass_oe',    'max'),
            ( 'pass_oe',    'sum'),
            ( 'pass_oe', 'median'),
            ( 'pass_oe',   'mean'),
            ( 'pass_oe',   'size'),
            ('pass_ypa',  'value')],
           length=1249)

AttributeError: 'DataFrame' object has no attribute 'success_rate'